In [2]:
import tensorflow as tf
import torch as torch
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv("~/PycharmProjects/Diabetes/finalNew.csv", sep=',')
Cholesterol_data = data[['Patient_ID', 'PerformedDate', 'Lipids_Res', 'Name_calc']]
Cholesterol_data = Cholesterol_data.dropna()
Cholesterol_data = Cholesterol_data[Cholesterol_data['Name_calc'] == 'Triglycerides (TG)']

Cholesterol_data = Cholesterol_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = Cholesterol_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Lipids_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Lipids_Res'] = grouped_lists['Lipids_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['Lipids_Res']]
print(data)

                                               Lipids_Res
0                           [65.0, 5.0, 41.0, 78.0, 48.0]
1                                          [221.0, 174.0]
2                              [145.0, 102.0, 68.0, 14.0]
3       [128.0, 129.0, 193.0, 131.0, 132.0, 163.0, 104...
4       [34.0, 262.0, 199.0, 299.0, 269.0, 272.0, 278....
...                                                   ...
122998                                            [412.0]
122999                                            [167.0]
123000                                     [337.0, 231.0]
123001                                            [232.0]
123002                                      [217.0, 22.0]

[123003 rows x 1 columns]


In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Lipids_Res'].at[index]) > 15 :
        seq = data['Lipids_Res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train, y_train, epochs=200, verbose=0)
#for index, row in data.iterrows() :
# demonstrate prediction

In [ ]:
for index, row in data.iterrows() :
    if len(data['Lipids_Res'].at[index]) > 15 :
        seq = data['Lipids_Res'].at[index]
        #x_input = np.asarray(X_test[-4:-1])
        #x_input = X_test.reshape((1, n_steps, n_features))
        yhat = model.predict(X_test, verbose=0)
        y_actual = np.asarray(y_test[-1:])
        model.evaluate(X_train, y_train)